# **0. 해커톤 진행 주의사항**

**1)  개발 관련 주의사항**
*   [1. 초기 환경 설정]은 절대 수정하지 말 것
*   모든 구현은 [2. 데이터 전처리] 및 [3.모델 생성]에서만 진행
*   [4. 모델 저장]에서 team_name 변수 변경 (예.`team_name = 'team01'`)
 *    트레이닝 중간에 checkpoint를 활용하여 모델을 저장한 경우에도 파일 이름 양식 통일 필수
*   Colab 사용중 실수로 데이터 손실이 발생할 수도 있으니 중간 결과값을 github에 업로드 
 *    "런타임->모든 런타임 재설정"은 절대 누르지 말 것 (저장한 모델 데이터가 모두 삭제됨)
*   효율적인 구현 및 테스팅을 위해 GPU 가속 기능 활성화
 *    "런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 설정"
*   주석을 최대한 자세히 작성
*   Keras API 관련하여 [Keras Documentation](https://keras.io/) 참조

**2) 제출 관련 주의사항**
*  제출물
 *  소스코드 (hackathon_teamXX.ipynb)
 *  컴파일된 모델 파일 (model_entire_teamXX.h5)
 *  모델 발표 자료 
* 제출 기한: **오후 6시 (단, 발표자료는 12시)**
* 제출 방법: [GitHub README](https://github.com/cauosshackathonta/2021_cau_oss_hackathon/) 참조

 
**3) 평가 관련 주의사항**
*  모델 성능 = 두개의 테스트 데이터 셋 분류 정확도에 대한 weighted sum
 *  model.evaluate(x_test1, y_test1) + model.evaluate(x_test2, y_test2) * 2
*  제출된 모델들의 테스트 데이터 셋 분류 정확도를 기준으로 수상작 결정
*  수상 후보들에 대해서는 소스코드를 기반으로 모델 재검증 
 
**4) 수상 실격 사유**
*  유사한 소스코드 or 알고리즘이 적발될 경우
*  소스코드와 제출된 모델이 상이한 경우
*  개발 관련 주의사항을 지키지 않은 경우
 *  예: [초기 환경 설정]을 수정한 경우
*  데이터 셋을 변조한 경우
 *  예: 테스트 데이터 셋을 트레이닝 데이터 셋에 포함하여 모델 생성
 *  단, tensorflow.data 및 dataset API를 사용하기 위해 변경하는 것은 허용. 이 경우, model evaluation 파트도 해당 API를 쓰도록 변경  
*  주석이 소스코드와 맞지 않거나 미비할 경우






# **1. 초기 환경 설정**



In [1]:
# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from keras.utils import np_utils

# 데이터셋 로드 (Training dataset: CIFAR10, test dataset: CIFAR10 & CIFAR10의 변형)
(x_train, y_train), (x_test1, y_test1) = keras.datasets.cifar10.load_data()

test_ds2 = tfds.load('cifar10_1/v6', split='test', shuffle_files=False, batch_size=-1)
test_ds2 = tfds.as_numpy(test_ds2)
x_test2, y_test2 = test_ds2['image'], test_ds2['label']

#분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test1 = np_utils.to_categorical(y_test1)
y_test2 = np_utils.to_categorical(y_test2)

# 총 클래스 개수
num_classes = y_train.shape[1]
input_shape = x_train.shape[1:]

Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


# **2. 데이터 전처리**



In [2]:
import numpy as np
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers

# 데이터 전처리 (예: normalization)
# 원본 데이터와 전처리 후 데이터를 구분하기 위해, 변수명 x_train_after, x_test1_after, x_test2_after를 변경하지 말 것
#x_train_after = x_train / 255.0
#x_test1_after = x_test1 / 255.0
#x_test2_after = x_test2 / 255.0

x_train_after = x_train
x_test1_after = x_test1
x_test2_after = x_test2

layer = preprocessing.Normalization()
layer.adapt(x_train_after)
layer.adapt(x_test1_after)
layer.adapt(x_test2_after)

print('x_train shape:', x_train_after.shape)
print('train samples', x_train_after.shape[0])
print('test1 samples', x_test1_after.shape[0])
print('test2 samples', x_test2_after.shape[0])
print('y_train shape:', y_train.shape)

print(type(x_train_after), type(x_test1_after), type(x_test2_after))

x_train shape: (50000, 32, 32, 3)
train samples 50000
test1 samples 10000
test2 samples 2000
y_train shape: (50000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


# **3. 모델 생성**

In [3]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=111)
indices = list(range(len(x_test1_after)))
for test_index, val_index in sss.split(indices, y_test1):
  x_test1_1000, y_test1_1000 = x_test1_after[val_index], y_test1[val_index]
x_test1_1000.shape

(1000, 32, 32, 3)

In [4]:
x_test_after = np.vstack([x_test2_after, x_test1_1000])
y_test= np.vstack([y_test2, y_test1_1000])
idx = range(len(y_test))
idx = np.random.permutation(idx)


In [19]:
data_augmentation = keras.Sequential(
    [
        preprocessing.RandomFlip("horizontal"),
        preprocessing.RandomRotation(0.2),
        preprocessing.RandomCrop(32, 32),
        preprocessing.RandomZoom(0.2, fill_mode='nearest'),
    ]
)

inputs = keras.Input(shape=input_shape)
x = data_augmentation(inputs)
#x = preprocessing.Rescaling(1.0 / 255)(x)
outputs = keras.applications.DenseNet169(
    weights=None, input_shape=input_shape, classes=num_classes
)(x)
model = keras.Model(inputs, outputs)

In [20]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/checkpoint_entire_best.h5', monitor='val_accuracy', verbose=1, save_weight_only=False, save_best_only=True, mode='auto')
callback = [es, cp_callback]

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.05,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=optimizer)
model.fit(x_train_after, y_train, batch_size=128, epochs=30, validation_data=(x_test_after[idx], y_test[idx]),callbacks=callback)

Epoch 1/30
391/391 [==============================] - 90s 191ms/step - loss: 2.0319 - accuracy: 0.3169 - val_loss: 2.0968 - val_accuracy: 0.2600

Epoch 00001: val_accuracy improved from -inf to 0.26000, saving model to /content/checkpoint_entire_best.h5
Epoch 2/30
391/391 [==============================] - 72s 183ms/step - loss: 1.6922 - accuracy: 0.4073 - val_loss: 1.8205 - val_accuracy: 0.3517

Epoch 00002: val_accuracy improved from 0.26000 to 0.35167, saving model to /content/checkpoint_entire_best.h5
Epoch 3/30
391/391 [==============================] - 72s 184ms/step - loss: 1.5517 - accuracy: 0.4526 - val_loss: 1.8129 - val_accuracy: 0.3717

Epoch 00003: val_accuracy improved from 0.35167 to 0.37167, saving model to /content/checkpoint_entire_best.h5
Epoch 4/30
391/391 [==============================] - 72s 183ms/step - loss: 1.4543 - accuracy: 0.4835 - val_loss: 1.7697 - val_accuracy: 0.3967

Epoch 00004: val_accuracy improved from 0.37167 to 0.39667, saving model to /content/c

# **4. 모델 저장**

In [17]:
save_path = '/content/'
team_name = 'team12'

# 트레이닝된 전체 모델을 저장합니다.
model.save(save_path +  'model_entire_'+ team_name + '.h5')

# **5. 모델 로드 및 평가**

In [18]:
save_path = '/content/'
team_name = 'team12'

model = keras.models.load_model(save_path + 'model_entire_' + team_name + '.h5')

model.evaluate(x_test1_after, y_test1)
model.evaluate(x_test2_after, y_test2)

63/63 [==============================] - 3s 47ms/step - loss: 2.7305 - accuracy: 0.4075


[2.73046612739563, 0.4074999988079071]